<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokaapi/exif_data_from_photos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Template Builder template: https://bit.ly/3mmyygl

In [9]:
from PIL import Image
import requests
import threading
import json
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO

In [6]:
TOLOKA_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
pool_id = '1111111'

In [10]:
def get_exif(filename):
    image = Image.open(filename)
    image.verify()

    return image._getexif()


def get_call(token: str, call: str, params: dict = {}) -> dict:
    headers = {
        "Authorization": "OAuth " + token
    }

    response = requests.get(url=f'https://sandbox.toloka.yandex.com/api/v1/{call}', headers=headers, params=params) #or production endpoint
    response.raise_for_status()

    return response


def get_attachments(pool_id: str) -> dict:
    param = {
        "pool_id": pool_id
        }
    response = get_call(token=TOLOKA_TOKEN, call=f'attachments', params=param)

    return response.json()


def get_attachment_ids(data: dict) -> dict:
    return {i['id']: i['name'] for i in data['items']}

def get_google_pos(cardinal_dir: tuple, deg: tuple, min: tuple, sec: tuple):
    g = '%C2%B0'
    s = '%22'

    lat = f"{deg[0]}{g}{min[0]}'{str(round(sec[0], 1))}{s}{cardinal_dir[0]}"
    lon = f"{deg[1]}{g}{min[1]}'{str(round(sec[1], 1))}{s}{cardinal_dir[1]}"

    return f"https://www.google.com/maps/place/{lat}+{lon}"


def coord_from_attachments(attachments: dict, path: str=''):
    for id in attachments.keys():
        with ThreadPoolExecutor() as e:
            future_data = e.submit(get_call, TOLOKA_TOKEN, f'attachments/{id}')
            future_attachment = e.submit(get_call, TOLOKA_TOKEN, f'attachments/{id}/download')
            data = future_data.result().json()
            attachment = future_attachment.result()
            
        exif = get_exif(BytesIO(attachment.content))

        cardinal_dir = (exif[34853][1], exif[34853][3])
        
        if '\x00' in cardinal_dir:
            continue

        deg = (exif[34853][2][0][0], exif[34853][4][0][0])
        min = (exif[34853][2][1][0], exif[34853][4][1][0])
        
        try:
            sec = (exif[34853][2][2][0] / exif[34853][2][2][1], exif[34853][4][2][0] / exif[34853][4][2][1])
            print('------------------------------------------')
            print(f"Assignment ID: {data['details']['assignment_id']}")
            print(f"User ID: {data['details']['user_id']}")
            print(f"Created: {data['created']}")
            print()
            print(f"Latitude: {deg[0]}° {min[0]}' {str(round(sec[0], 5)).ljust(8, '0')}\" {cardinal_dir[0]}")
            print(f"Longitude: {deg[1]}° {min[1]}' {str(round(sec[1], 5)).ljust(8, '0')}\" {cardinal_dir[1]}")
            print(get_google_pos(cardinal_dir, deg, min, sec))
            print()
        except ZeroDivisionError:
            pass

In [ ]:
json_data = get_attachments(pool_id=pool_id)
attachments = get_attachment_ids(json_data)
coord_from_attachments(attachments=attachments)